<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine_tuning_mixed/ask_3_mixed_pubmed_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (60) SSL certificate problem: certificate has expired
More details here: https://curl.haxx.se/docs/sslcerts.html

curl failed to verify the legitimacy of the server and therefore could not
establish a secure connection to it. To learn more about this situation and
how to fix it, please visit the web page mentioned above.


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-3.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-3.validation.csv",header=None)
valid.head()

,0,1
0,2,left heel pain
1,2,right heel became painful
2,8,Arthritis
3,8,arthritis
4,8,arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-3.test.csv",header=None)
test.head()

,0,1
0,2,heel pain
1,5,rapid heart beat
2,6,diastolic dysfunction
3,8,arthritis
4,8,arthritis


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_mixed.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 '.',
 'was',
 'with',
 'i',
 'diclofenac',
 ',',
 'for',
 'were',
 'on',
 'sodium',
 'pain',
 'that',
 'by',
 'is',
 'this',
 'drug',
 'as',
 'my',
 'it',
 'patients',
 'at',
 'mg',
 'or',
 'after',
 'from',
 'have',
 'not',
 'group',
 'treatment',
 'study',
 'be',
 'but',
 'lipitor',
 'an',
 'effects',
 'p',
 'release',
 'no',
 'had',
 'are',
 'than',
 'all',
 'effect',
 'two',
 'day',
 'compared',
 'been',
 'both',
 'using',
 'time',
 'groups',
 'drugs',
 'which',
 'days',
 'taking',
 'also',
 'has',
 'dose',
 'these',
 'significantly',
 'significant',
 'inflammatory',
 'h',
 'side',
 'more',
 'one',
 'between',
 'results',
 'voltaren',
 'used',
 'anti',
 'cholesterol',
 'placebo',
 'potassium',
 'muscle',
 'we',
 'when',
 'did',
 'me',
 'ds',
 'efficacy',
 'years',
 'showed',
 'other',
 'treated',
 'ml',
 'over',
 '-',
 'high',
 'different',
 'there',

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.218214,4.662446,0.234365,15:54


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.513160,4.379565,0.257667,15:56


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.216438,4.343594,0.260850,15:52


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.176575,4.318535,0.262995,15:52


In [0]:
learn.save_encoder('bigtweet_fold0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.703268,3.367118,0.414758,01:10


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.708825,2.833122,0.527990,01:22


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.069316,2.527505,0.569975,03:25


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.821660,2.233130,0.614504,03:18
1,1.680130,2.110884,0.627226,03:34


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.482541,1.924985,0.665394,03:30
1,1.266917,1.816185,0.678117,03:34


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.150562,1.723815,0.698473,03:39
1,1.066399,1.636857,0.713740,03:24


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.964710,1.569510,0.734097,03:24
1,0.871141,1.516021,0.739186,03:30


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.810887,1.474881,0.734097,03:39
1,0.746934,1.437652,0.748092,03:32


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.753493,1.397306,0.758270,03:40
1,0.709777,1.396966,0.756997,03:30
2,0.623261,1.330490,0.763359,03:26
3,0.549349,1.324307,0.768448,03:34


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.546781,1.348784,0.759542,03:21
1,0.558520,1.363443,0.765903,03:24
2,0.481616,1.342639,0.763359,03:15
3,0.462867,1.316632,0.772265,03:25


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.467077,1.347968,0.770992,03:29
1,0.509597,1.353134,0.773537,03:22
2,0.453180,1.352646,0.773537,03:26
3,0.392872,1.359782,0.776081,03:31


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.415284,1.381698,0.783715,03:16
1,0.452302,1.338662,0.778626,03:30
2,0.413590,1.348775,0.786260,03:34
3,0.355991,1.306787,0.782443,03:31


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.377434,1.395419,0.778626,03:19
1,0.408532,1.388490,0.777354,03:37
2,0.365455,1.350533,0.786260,03:21
3,0.332749,1.355672,0.787532,03:26


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.363450,1.430044,0.777354,03:22
1,0.376261,1.444160,0.774809,03:31
2,0.363610,1.448026,0.779898,03:23
3,0.308315,1.400187,0.783715,03:30


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.321751,1.392469,0.778626,03:23
1,0.366012,1.520175,0.776081,03:18
2,0.359271,1.421839,0.792621,03:37
3,0.307351,1.470095,0.790076,03:18


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.316123,1.470673,0.776081,03:37
1,0.314682,1.429124,0.779898,03:23


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.328290,1.486013,0.787532,03:22
1,0.318161,1.398489,0.786260,03:31


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.346107,1.408137,0.786260,03:42


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[2, 5, 6, 8, 8, 8, 12, 182, 330, 27, 27, 662, 29, 30, 31, 34, 877, 1005, 37, 37, 37, 37, 37, 37, 37, 37, 289, 49, 325, 49, 537, 49, 473, 49, 775, 100, 100, 100, 100, 100, 53, 53, 53, 60, 61, 61, 491, 62, 100, 225, 205, 71, 107, 179, 76, 652, 228, 41, 79, 79, 79, 79, 368, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 289, 614, 614, 289, 80, 80, 80, 89, 597, 537, 91, 94, 94, 360, 135, 94, 135, 135, 135, 135, 135, 135, 135, 135, 94, 135, 761, 761, 98, 98, 848, 100, 100, 100, 100, 538, 104, 644, 877, 108, 108, 108, 289, 111, 289, 189, 117, 775, 535, 436, 468, 537, 125, 125, 125, 125, 125, 125, 125, 289, 343, 82, 343, 343, 131, 473, 473, 473, 140, 226, 447, 151, 809, 475, 626, 73, 163, 163, 167, 49, 644, 604, 172, 289, 260, 172, 172, 427, 115, 264, 182, 182, 289, 289, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 187, 187, 187, 187, 187, 187, 240, 190, 190, 190, 190, 637, 192, 105, 105, 105, 197, 197,

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

580
0.6697459584295612
